In [1]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [2]:
# !mkdir -p drive
# !google-drive-ocamlfuse drive

In [1]:
import os 
import sys
import tensorflow as tf
import pandas as pd

from tensorflow.python.keras import models
from sklearn import model_selection
from __future__ import absolute_import

# os.chdir("/content/drive/Hamming")
# sys.path.append("Hamming")
# import functions as f
# import model_lib 

In [2]:
tf.test.gpu_device_name()

''

In [3]:
!ls

dataset_files  hamming-codes.ipynb  README.md
functions.py   model_lib.py	    visualisation.ipynb


In [4]:
# !git pull

In [5]:
TRAIN_PATH = './dataset_files/hamming.txt'  # "/content/drive/Hamming/dataset_files/hamming.txt"

COLUMN_NAMES = ['id', 'plainword', 'codeword', 
                'id_error', 'bin_error', 'defective_codeword']

In [6]:
def hamming_distance(first: str, second: str) -> int:
    return len([1 for (x, y) in zip(first, second) if x != y])

In [7]:
def loss_hamming(y, y_pred):  # , w):
    return sum(1 for (a,b) in zip(y, y_pred) if y != y_pred) / y.shape[0] # надо ли делить????

In [8]:
def load_data():
    data = pd.read_csv(TRAIN_PATH, sep=';', names=COLUMN_NAMES)

    # make features
    data['dec_defective_codeword'] = data['defective_codeword'][:].apply(lambda x: int(x, 2))
    for j in range(len(data['defective_codeword'][0])):
        data['bin_' + str(j)] = data['defective_codeword'][:].apply(lambda x: int(x[j]))
    data['dec_plainword'] = data['plainword'][:].apply(lambda x: int(x, 2))
    return data

In [9]:
def split_data(test_size):
    train_data, test_data, train_labels, test_labels = \
    model_selection.train_test_split(data.loc[:, 'bin_0':'bin_30'], 
                                        data['id_error'], 
                                        test_size = test_size)
    return train_data, test_data, train_labels, test_labels

In [10]:
data = load_data() #pd.read_csv(functions.TRAIN_PATH, sep=';', names = functions.COLUMN_NAMES)

In [11]:
data.head()

,id,plainword,codeword,id_error,bin_error,defective_codeword,dec_defective_codeword,bin_0,bin_1,bin_2,...,bin_22,bin_23,bin_24,bin_25,bin_26,bin_27,bin_28,bin_29,bin_30,dec_plainword
0,0,10110000001111101100000100,0001001100000011111011000001000,8,0000000010000000000000000000000,0001001110000011111011000001000,163706376,0,0,0,...,0,0,0,0,0,1,0,0,0,46201604
1,1,00101011010000101000111100,1001101010110100001010001111000,17,0000000000000000010000000000000,1001101010110100011010001111000,1297757304,1,0,0,...,0,0,1,1,1,1,0,0,0,11340348
2,2,10010100110111111111111100,1101100101001101111111111111000,18,0000000000000000001000000000000,1101100101001101110111111111000,1822879736,1,1,0,...,1,1,1,1,1,1,0,0,0,39026684
3,3,00011000001101000100010111,1011100110000011010001000101110,13,0000000000000100000000000000000,1011100110000111010001000101110,1556324910,1,0,1,...,0,0,0,1,0,1,1,1,0,6344983
4,4,10011101101000001100111011,0011000111011010000011001110110,19,0000000000000000000100000000000,0011000111011010000111001110110,418188918,0,0,1,...,0,0,1,1,1,0,1,1,0,41321275


In [37]:
def make_features():
    data['dec_defective_codeword'] = data['defective_codeword'][:].apply(lambda x: int(x, 2))

    for j in range(len(data['defective_codeword'][0])):
        data['bin_' + str(j)] = data['defective_codeword'][:].apply(lambda x: int(x[j]))
  
    data['dec_plainword'] = data['plainword'][:].apply(lambda x: int(x, 2))

In [13]:
make_features()

In [14]:
data.head()

,id,plainword,codeword,id_error,bin_error,defective_codeword,dec_defective_codeword,bin_0,bin_1,bin_2,...,bin_22,bin_23,bin_24,bin_25,bin_26,bin_27,bin_28,bin_29,bin_30,dec_plainword
0,0,10110000001111101100000100,0001001100000011111011000001000,8,0000000010000000000000000000000,0001001110000011111011000001000,163706376,0,0,0,...,0,0,0,0,0,1,0,0,0,46201604
1,1,00101011010000101000111100,1001101010110100001010001111000,17,0000000000000000010000000000000,1001101010110100011010001111000,1297757304,1,0,0,...,0,0,1,1,1,1,0,0,0,11340348
2,2,10010100110111111111111100,1101100101001101111111111111000,18,0000000000000000001000000000000,1101100101001101110111111111000,1822879736,1,1,0,...,1,1,1,1,1,1,0,0,0,39026684
3,3,00011000001101000100010111,1011100110000011010001000101110,13,0000000000000100000000000000000,1011100110000111010001000101110,1556324910,1,0,1,...,0,0,0,1,0,1,1,1,0,6344983
4,4,10011101101000001100111011,0011000111011010000011001110110,19,0000000000000000000100000000000,0011000111011010000111001110110,418188918,0,0,1,...,0,0,1,1,1,0,1,1,0,41321275


In [15]:
train_data, test_data, train_labels, test_labels = split_data(test_size=0.3)

In [16]:
# Create feature columns for all features.
feature_columns = []
for key in train_data.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))

In [17]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[100, 100, 100],
    n_classes=train_data.shape[1]
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp3x2y586u', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2af351e400>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [26]:
train_input_fn = tf.estimator.inputs.pandas_input_fn(x=train_data,
                                                    y=train_labels,
                                                    num_epochs=3,
                                                    batch_size=100,
                                                    shuffle=True
                                                    )

In [19]:
test_input_fn = tf.estimator.inputs.pandas_input_fn(x=test_data,
                                                    y=test_labels,
                                                    num_epochs=3,
                                                    batch_size=100,
                                                    shuffle=True
                                                    )

In [31]:
classifier.train(input_fn=train_input_fn, steps=10**6)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp3x2y586u/model.ckpt-4131
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 4132 into /tmp/tmp3x2y586u/model.ckpt.
INFO:tensorflow:loss = 335.69876, step = 4132
INFO:tensorflow:global_step/sec: 142.887
INFO:tensorflow:loss = 331.0602, step = 4232 (0.701 sec)
INFO:tensorflow:global_step/sec: 206.821
INFO:tensorflow:loss = 331.33496, step = 4332 (0.484 sec)
INFO:tensorflow:global_step/sec: 165.227
INFO:tensorflow:loss = 336.47205, step = 4432 (0.605 sec)
INFO:tensorflow:global_step/sec: 192.903
INFO:tensorflow:loss = 333.5416, step = 4532 (0.518 sec)
INFO:tensorflow:global_step/sec: 205.571
INFO:tensorflow:loss = 329.67706, step = 4632 (0.486 sec)
INFO:tensorflow:global_step/sec: 157.312
INFO:tensorflow:loss = 328.291

In [32]:
accuracy_score = classifier.evaluate(input_fn=test_input_fn)['accuracy']
print('\nTest Accuracy: {0:f}%\n'.format(accuracy_score*100))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-03-18-12:54:44
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp3x2y586u/model.ckpt-5508
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-03-18-12:54:47
INFO:tensorflow:Saving dict for global step 5508: accuracy = 0.030822441, average_loss = 3.4761152, global_step = 5508, loss = 347.51135

Test Accuracy: 3.082244%



In [44]:
def model_fn_2(features, labels, mode, params):
    h0 = features

    h1 = tf.layers.dense(h0, 100, activation=tf.nn.relu, name="h1")
    h2 = tf.layers.dense(h1, 100, activation=tf.nn.relu, name="h2")
    h3 = tf.layers.dense(h2, 100, activation=tf.nn.relu, name="h3")

    assignment_map = {
        'h1/': 'h1/',
        'h2/': 'h2/'
    }
    tf.train.init_from_checkpoint('mnist_1', assignment_map)

    dnn_prediction = tf.layers.dense(h3, 10, name="dnn_prediction")

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, predictions=dnn_prediction)

    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

# Estimator 2: three hidden layers
estimator_2 = tf.estimator.Estimator(model_fn_2, model_dir=TRAIN_PATH)

estimator_2.train(input_fn=train_input_fn, steps=1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './dataset_files/hamming.txt', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2af21c6dd8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.


AttributeError: 'dict' object has no attribute 'dtype'

In [39]:
train_input_fn

<function tensorflow.python.estimator.inputs.pandas_io.pandas_input_fn.<locals>.input_fn>